In [1]:
# train.py
import random
import numpy as np
import torch
from transformers import BertConfig, Trainer, TrainingArguments, EarlyStoppingCallback, EvalPrediction
from seqeval.metrics import classification_report
from transformers.trainer_utils import EvalLoopOutput
from transformers import AutoTokenizer, get_scheduler
from bert_crf_data_processing import prepare_datasets, generate_label_map
from bert_crf_model import BERT_CRF
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix
import pandas as pd
import time  # 添加 time 模块用于计时

# 生成标签映射
label_map = generate_label_map()
label_map_inv = {v: k for k, v in label_map.items()}

class Config:
    # 设置随机数种子
    seed_val = 42
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    max_length = 512
    batch_size = 32
    num_labels = 13  
    model_name = "../models/bert_pretrained"
    data_path = "../datasets/train/trainset_B.txt"
    output_dir = "../../../hy-tmp/models/Ablation_bertlr5e-5_crflr5e-3_cosine"  # 修改输出目录

    num_epochs = 35
    weight_decay = 0.01
    bert_lr = 5e-5  # BERT learning rate
    crf_lr = 5e-3   # CRF learning rate
    learning_rate = 5e-5  # 额外学习率参数，如果需要可以调整
    warmup_ratio = 0.1

    lr_scheduler_type = "cosine"
    early_stopping_patience = 7
    max_grad_norm = 1.0

class NERTrainer(Trainer):
    def __init__(self, label_map_inv, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.label_map_inv = label_map_inv

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs.get("loss")
            decoded_tags = model.crf.decode(outputs["logits"], mask=inputs["attention_mask"].bool())
        return (loss, decoded_tags, inputs["labels"].cpu().numpy())

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        loss = outputs["loss"]
        return (loss, outputs) if return_outputs else loss

    def evaluation_loop(self, dataloader, description, prediction_loss_only=False, ignore_keys=None, metric_key_prefix="eval"):
        model = self.model.eval()
        total_loss = 0.0
        all_preds = []
        all_labels = []
        all_masks = []  # 新增列表保存所有attention_mask

        for batch in dataloader:
            inputs = self._prepare_inputs(batch)
            with torch.no_grad():
                loss, decoded_tags, labels = self.prediction_step(model, inputs, prediction_loss_only)
                total_loss += loss.item()
                all_preds.extend(decoded_tags)
                all_labels.extend(labels)
                all_masks.extend(inputs["attention_mask"].cpu().numpy())  # 保存所有mask

        # 处理时使用所有保存的mask
        true_labels = []
        pred_labels = []
        for pred_seq, true_seq, mask in zip(all_preds, all_labels, all_masks):
            seq_len = np.sum(mask)
            true_labels.append([self.label_map_inv[l] for l in true_seq[:seq_len]])
            pred_labels.append([self.label_map_inv[p] for p in pred_seq[:seq_len]])

        report = classification_report(true_labels, pred_labels, output_dict=True)
        # 输出指标查看
        print(classification_report(true_labels, pred_labels))

        metrics = {
            f"{metric_key_prefix}_loss": total_loss / len(dataloader),
            f"{metric_key_prefix}_precision": report["micro avg"]["precision"],
            f"{metric_key_prefix}_recall": report["micro avg"]["recall"],
            f"{metric_key_prefix}_f1": report["micro avg"]["f1-score"],
        }

        # 生成混淆矩阵 (绘制图像并保存)
        true_entities = []
        pred_entities = []
        for tl_seq, pl_seq in zip(true_labels, pred_labels):
            for tl, pl in zip(tl_seq, pl_seq):
                if tl == 'O':
                    true_ent = 'O'
                else:
                    parts = tl.split('-', 1)
                    true_ent = parts[1] if len(parts) > 1 else 'O'
                if pl == 'O':
                    pred_ent = 'O'
                else:
                    parts = pl.split('-', 1)
                    pred_ent = parts[1] if len(parts) > 1 else 'O'
                true_entities.append(true_ent)
                pred_entities.append(pred_ent)

        entity_types = sorted(list(set(true_entities + pred_entities)))
        cm = confusion_matrix(true_entities, pred_entities, labels=entity_types)
        cm_df = pd.DataFrame(cm, index=entity_types, columns=entity_types)

        # ---  绘制混淆矩阵热图 ---
        fig = plt.figure(figsize=(12, 10))  # 可调整图像大小
        ax = fig.add_subplot(111)
        cax = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)  # 使用蓝色的 colormap
        fig.colorbar(cax)  # 添加颜色栏
        tick_marks = np.arange(len(entity_types))
        ax.set_xticks(tick_marks)
        ax.set_yticks(tick_marks)
        ax.set_xticklabels(entity_types, rotation=45, ha="right")  # 旋转 x 轴标签，使其不重叠
        ax.set_yticklabels(entity_types)

        # 在混淆矩阵上显示数值
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, cm[i, j], ha="center", va="center", color='black' if cm[i,j] < cm.max()/2 else 'white')  # 根据数值大小调整颜色

        plt.tight_layout()  # 自动调整子图参数，提供图像边缘周围的填充
        plt.ylabel('True Entity')
        plt.xlabel('Predicted Entity')
        plt.title(f'Confusion Matrix - Epoch {int(self.state.epoch)}')  # 添加标题

        # 获取当前epoch数
        epoch_num = int(self.state.epoch)
        save_dir = os.path.join(self.args.output_dir, 'confusion_matrices')
        os.makedirs(save_dir, exist_ok=True)
        filename = f"confusion_matrix_epoch_{epoch_num}.png"  # 保存为 PNG 图片
        filepath = os.path.join(save_dir, filename)
        plt.savefig(filepath)  # 保存图像
        plt.close(fig)  # 关闭图像，释放内存
        print(f"\nConfusion matrix image for epoch {epoch_num} saved to {filepath}")

        return EvalLoopOutput(
            predictions=all_preds,
            label_ids=all_labels,
            metrics=metrics,
            num_samples=len(all_labels),
        )

def collate_fn(batch):
    return {
        "input_ids": torch.stack([x["input_ids"] for x in batch]),
        "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
        "labels": torch.stack([x["labels"] for x in batch]),
    }

def main():
    cfg = Config()
    random.seed(cfg.seed_val)
    np.random.seed(cfg.seed_val)
    torch.manual_seed(cfg.seed_val)
    torch.cuda.manual_seed_all(cfg.seed_val)

    # 初始化模型
    pre_config = BertConfig.from_pretrained(cfg.model_name, num_labels=cfg.num_labels)
    model = BERT_CRF.from_pretrained(cfg.model_name, config=pre_config).to(cfg.device)

    # 加载Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
    train_ds, val_ds = prepare_datasets(cfg.data_path, tokenizer)

    optimizer_grouped_parameters = [
        {
            "name": "bert_base",
            "params": [p for n, p in model.bert.named_parameters() if p.requires_grad],
            "lr": cfg.bert_lr,
        },
        {
            "name": "crf_base",
            "params": [p for n, p in model.crf.named_parameters() if p.requires_grad],
            "lr": cfg.crf_lr,
        },
        {
            "params": [p for n, p in model.classifier.named_parameters() if p.requires_grad],  # 假设分类器层名为 classifier
            "lr": cfg.learning_rate,  # 使用 learning_rate
        }
    ]
    # 创建优化器，使用 AdamW，并将分组参数传递进去
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, weight_decay=cfg.weight_decay)

    num_training_steps = len(train_ds) // cfg.batch_size * cfg.num_epochs  # 粗略估计，根据实际情况调整
    num_warmup_steps = int(cfg.warmup_ratio * num_training_steps)

    lr_scheduler = get_scheduler(
        name=cfg.lr_scheduler_type,  # 可以尝试 "cosine"
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
    )

    # 训练参数
    training_args = TrainingArguments(
        output_dir=cfg.output_dir,
        per_device_train_batch_size=cfg.batch_size,
        per_device_eval_batch_size=cfg.batch_size * 2,
        num_train_epochs=cfg.num_epochs,
        weight_decay=cfg.weight_decay,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        remove_unused_columns=False,
        fp16=False,
        max_grad_norm=cfg.max_grad_norm,
        logging_steps=20,
        seed=cfg.seed_val,
        data_seed=cfg.seed_val,  # 控制数据加载的随机性，保证实验可复现
    )

    trainer = NERTrainer(
        label_map_inv=label_map_inv,
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=None,
        data_collator=collate_fn,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.early_stopping_patience)],  # 确保早停回调已开启
        optimizers=(optimizer, lr_scheduler),
    )

    # 计算训练时间
    print("Starting training...")
    start_time = time.time()  # 记录开始时间

    # 存储训练过程中的 loss 和 metrics
    train_history = trainer.train()

    end_time = time.time()  # 记录结束时间
    training_time = end_time - start_time  # 计算总训练时间（秒）

    # 转换为小时、分钟、秒格式
    hours = int(training_time // 3600)
    minutes = int((training_time % 3600) // 60)
    seconds = training_time % 60
    print(f"Training completed in: {hours} hours, {minutes} minutes, {seconds:.2f} seconds (Total: {training_time:.2f} seconds)")

    # 从 trainer 的 state 中获取日志信息
    log_history = trainer.state.log_history

    # 提取训练 loss 和 验证 metrics
    train_losses = []
    eval_losses = []
    eval_f1s = []
    eval_precisions = []
    eval_recalls = []
    epochs = []
    logged_epochs = set()

    for log_data in log_history:
        if 'loss' in log_data:
            train_losses.append(log_data['loss'])
            epoch_num = log_data['epoch']
            if epoch_num not in logged_epochs:
                epochs.append(epoch_num)
                logged_epochs.add(epoch_num)
        if 'eval_loss' in log_data:
            eval_losses.append(log_data['eval_loss'])
            epoch_num = log_data['epoch']
            if epoch_num not in logged_epochs:
                epochs.append(epoch_num)
                logged_epochs.add(epoch_num)
        if 'eval_f1' in log_data:
            eval_f1s.append(log_data['eval_f1'])
        if 'eval_precision' in log_data:
            eval_precisions.append(log_data['eval_precision'])
        if 'eval_recall' in log_data:
            eval_recalls.append(log_data['eval_recall'])

    epochs = sorted(list(set(epochs)))

    model.save_pretrained(cfg.output_dir)
    tokenizer.save_pretrained(cfg.output_dir)

    final_eval_metrics = trainer.state.log_history[-1]
    final_report_string = final_eval_metrics.get("eval_report_string")
    if final_report_string:
        print("\nFinal Evaluation Report:\n", final_report_string)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BERT_CRF were not initialized from the model checkpoint at ../models/bert_pretrained and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'crf.end_transitions', 'crf.start_transitions', 'crf.transitions']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-03-06 22:38:32 - INFO - 数据集划分结果：
- 总序列数：181
- 训练集序列数：144
- 验证集序列数：37
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


训练集标签分布: Counter({0: 57090, 1: 2530, 3: 2530, 7: 2178, 5: 2177, 11: 1765, 9: 1764, 2: 1026, 6: 523, 8: 403, 4: 378, 10: 337, 12: 292})
验证集标签分布: Counter({0: 14722, 1: 693, 3: 693, 5: 565, 7: 564, 9: 436, 11: 435, 2: 295, 8: 120, 4: 100, 6: 94, 10: 81, 12: 72})
Starting training...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,265672029379088028376219204452352.000000,0.007465,0.051811,0.013050
2,No log,265652106281580779287420046671872.000000,0.000000,0.000000,0.000000
3,No log,265617966216434862159526344261632.000000,0.000000,0.000000,0.000000
4,265645870158632871978525838540800.000000,265578158707046591650061619298304.000000,0.046512,0.005030,0.009079
5,265645870158632871978525838540800.000000,265538738053920597821932800311296.000000,0.543590,0.586519,0.564239
6,265645870158632871978525838540800.000000,265499936370814246683941430886400.000000,0.671910,0.752012,0.709708
7,265645870158632871978525838540800.000000,265462101828363587249289826402304.000000,0.735063,0.798290,0.765373
8,265504373612142567424623676227584.000000,265425466540325985526779530444800.000000,0.811275,0.832495,0.821748
9,265504373612142567424623676227584.000000,265390359334524376695546063093760.000000,0.831041,0.851107,0.840954
10,265504373612142567424623676227584.000000,265356934953463671428123786739712.000000,0.831150,0.861670,0.846135


              precision    recall  f1-score   support

          NR       0.00      0.01      0.01       794
          NS       0.01      0.12      0.02       685
           T       0.01      0.02      0.01       509

   micro avg       0.01      0.05      0.01      1988
   macro avg       0.01      0.05      0.01      1988
weighted avg       0.01      0.05      0.01      1988


Confusion matrix image for epoch 1 saved to ../../../hy-tmp/models/Ablation_bertlr5e-5_crflr5e-3_cosine/confusion_matrices/confusion_matrix_epoch_1.png


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          NR       0.00      0.00      0.00       794
          NS       0.00      0.00      0.00       685
           T       0.00      0.00      0.00       509

   micro avg       0.00      0.00      0.00      1988
   macro avg       0.00      0.00      0.00      1988
weighted avg       0.00      0.00      0.00      1988


Confusion matrix image for epoch 2 saved to ../../../hy-tmp/models/Ablation_bertlr5e-5_crflr5e-3_cosine/confusion_matrices/confusion_matrix_epoch_2.png


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          NR       0.00      0.00      0.00       794
          NS       0.00      0.00      0.00       685
           T       0.00      0.00      0.00       509

   micro avg       0.00      0.00      0.00      1988
   macro avg       0.00      0.00      0.00      1988
weighted avg       0.00      0.00      0.00      1988


Confusion matrix image for epoch 3 saved to ../../../hy-tmp/models/Ablation_bertlr5e-5_crflr5e-3_cosine/confusion_matrices/confusion_matrix_epoch_3.png
              precision    recall  f1-score   support

          NR       0.01      0.00      0.00       794
          NS       0.00      0.00      0.00       685
           T       0.09      0.02      0.03       509

   micro avg       0.05      0.01      0.01      1988
   macro avg       0.03      0.01      0.01      1988
weighted avg       0.03      0.01      0.01      1988


Confusion matrix image for epoch 4 saved to ../../../hy-tmp/models/Ablation_bertlr5e